In [ ]:
import json
import boto3
import urllib3
import datetime

# Replace with your Data Firehose name
FIREHOSE_NAME = 'PUT-S3-n224V'

# API details
API_URL = 'https://api.coresignal.com/cdapi/v1/professional_network/job/search/filter'
API_KEY = top_secrets

def lambda_handler(event, context):
    http = urllib3.PoolManager()
    firehose_client = boto3.client('firehose')

    try:
        # Make the API request
        headers = {
            'Authorization': f'Bearer {API_KEY}',
            'Content-Type': 'application/json'
        }

        payload = {
            "application_active": True,
            "last_updated_gte": "2024-12-01 00:00:01",
            "country": "United States", 
            "title": "(Program manager) AND (Remote)",
            "deleted": 0,
            "application_active": 1,
            "industry": "Software development"
        }

        response = http.request(
            'POST',
            API_URL,
            body=json.dumps(payload),
            headers=headers
        )

        # Parse the API response
        if response.status == 200:
            data = json.loads(response.data.decode('utf-8'))

            if isinstance(data, list):
                for record in data:
                    if isinstance(record, dict):
                        # Add timestamp to dictionary records
                        record['timestamp'] = datetime.datetime.utcnow().isoformat()
                        send_to_firehose(firehose_client, record)
                    elif isinstance(record, int):
                        # Convert integer to JSON-compatible dictionary
                        json_record = {
                            "value": record,
                            "timestamp": datetime.datetime.utcnow().isoformat()
                        }
                        send_to_firehose(firehose_client, json_record)
                    else:
                        # Log unsupported record types
                        print(f"Skipping unsupported record type: {type(record)}, Value: {record}")
            elif isinstance(data, dict):
                data['timestamp'] = datetime.datetime.utcnow().isoformat()
                send_to_firehose(firehose_client, data)
            else:
                print(f"Unexpected data format: {type(data)}, Value: {data}")
                raise ValueError("Unexpected API response format")

            return {
                'statusCode': 200,
                'body': json.dumps({'message': 'Data successfully sent to Firehose'})
            }
        else:
            return {
                'statusCode': response.status,
                'body': f"API request failed: {response.data.decode('utf-8')}"
            }

    except Exception as e:
        return {
            'statusCode': 500,
            'body': f"An error occurred: {str(e)}"
        }

def send_to_firehose(client, record):
    """
    Helper function to send data to Firehose
    """
    client.put_record(
        DeliveryStreamName=FIREHOSE_NAME,
        Record={
            'Data': json.dumps(record) + '\n'  # Firehose requires newline-delimited JSON
        }
    )